In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("./data/train_dataset.csv")

data['is_test'] = False
evaluation = pd.read_csv("./data/evaluation_public.csv")
evaluation['is_test'] = True
sample = pd.read_csv("./data/sample_submission.csv")

all_data = pd.concat([data, evaluation]).reset_index(drop=True)

# 特征处理

In [3]:
all_data['date'] = pd.to_datetime(all_data['time'])
all_data['hour'] = all_data['date'].dt.hour
all_data['year'] = all_data['date'].dt.year
all_data['month'] = all_data['date'].dt.month
all_data['minute'] = all_data['date'].dt.minute
all_data['weekday'] = all_data['date'].dt.weekday
all_data['day'] = all_data['date'].dt.day
all_data['hour'] = all_data['date'].dt.hour
all_data['ts'] = all_data['hour']*3600 + all_data['minute']*60 + all_data['date'].dt.second

In [4]:
import gc
features = [ 'JS_NH3', 'CS_NH3', 'JS_TN', 'CS_TN', 'JS_LL', 'CS_LL', 'MCCS_NH4', 'MCCS_NO3', 'JS_COD', 'CS_COD', 'JS_SW', 'CS_SW', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL', 'B_QY_ORP', 'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_JS_DO', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP','weekday','hour', 'ts']
features = [f for f in features if f not in ['time', 'Label1', 'Label2','Label1_log','Label2_log','CS_LL',
                                             'CS_NH3', 'JS_SW'
                                             ]]

labels = ['Label1', 'Label2']
train = all_data[~all_data['is_test']].copy(deep=True)
test = all_data[all_data['is_test']].copy(deep=True)
test['is_train'] = False
train['is_train'] = True
data = pd.concat([train, test]).reset_index(drop=True)
del all_data
gc.collect()

42

In [5]:
df_list = []
i = 0
add_featuers = []
length = 0
for f in ['JS_NH3', 'CS_TN', 'JS_LL', 'JS_COD', 'CS_COD', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL',
          'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP']:
    for r in [15]:
        train_rolling = data[f].rolling(window=r, center=False)
        f_mean_name = 'rolling{}_{}_mean'.format(r,f)
        f_max_name = 'rolling{}_{}_max'.format(r,f)
        f_min_name = 'rolling{}_{}_min'.format(r,f)
        f_std_name = 'rolling{}_{}_std'.format(r,f)
        data[f_mean_name] = train_rolling.mean().fillna(0).values
        data[f_max_name] = train_rolling.max().fillna(0).values
        data[f_min_name] = train_rolling.min().fillna(0).values
        data[f_std_name] = train_rolling.std().fillna(0).values
        data[f'{f}_{f_mean_name}_cha'] = data[f] - data[f_mean_name]
        data[f'{f}_{f_max_name}_cha'] = data[f] - data[f_max_name]
        data[f'{f}_{f_min_name}_cha'] = data[f] - data[f_min_name]
        if i == 0:
            add_featuers.append(f_mean_name)
            add_featuers.append(f_max_name)
            add_featuers.append(f_min_name)
            add_featuers.append(f_std_name)
            add_featuers.append(f'{f}_{f_mean_name}_cha')
            add_featuers.append(f'{f}_{f_max_name}_cha')
            add_featuers.append(f'{f}_{f_min_name}_cha')
features.extend(add_featuers)

# 对所有的特征进行划分
for f in features:
    if f not in ['weekday','hour', 'ts']:
        q = len(data[f].drop_duplicates())
        data[f] = pd.qcut(data[f], q=int(q/10), labels=False, duplicates="drop")

train = data[data['is_train']].reset_index(drop=True)
test = data[~data['is_train']].reset_index(drop=True)

C:\Users\38125\AppData\Local\Temp/ipykernel_17044/88824995.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{f}_{f_mean_name}_cha'] = data[f] - data[f_mean_name]
C:\Users\38125\AppData\Local\Temp/ipykernel_17044/88824995.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{f}_{f_max_name}_cha'] = data[f] - data[f_max_name]
C:\Users\38125\AppData\Local\Temp/ipykernel_17044/88824995.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whic

# 删除分布不均衡的特征

In [6]:
train = train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)
train['Label1'] = np.log1p(train['Label1'])
train['Label2'] = np.log1p(train['Label2'])

In [7]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
score_list = []
for label in labels:
    train[label] = train[label].fillna(0)
    tmp = train[~train[label].isna()].reset_index(drop=True).copy(deep=True)
    print(len(tmp))
    print(np.max(tmp[label]), np.min(tmp[label]))
    test_size = 4000
    X_train = tmp[features][:-test_size]
    X_test = tmp[features][-test_size:]
    y_train = tmp[label][:-test_size]
    y_test = tmp[label][-test_size:]
    if label == "Label1":
        model = lgb.LGBMRegressor(
            boosting="gbdt",
            max_depth=4,
            learning_rate=0.1,
            n_estimators=10000,
            min_child_weight=1,
            min_data_in_leaf=50,
            subsample = 0.8,
            feature_fraction=0.4,
            bagging_seed=1,
            reg_alpha=1,
            reg_lambda=1,  # 此处不改了
            min_sum_hessian_in_leaf=0.01,
            random_state=1212
        )
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['mse'],
              early_stopping_rounds=150, verbose=200)
        test_pred = model.predict(X_test)
        score_list.append(np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(test_pred))))
        print(f"score_list = {score_list}")
        model2 = lgb.LGBMRegressor(
            boosting="gbdt",
            max_depth=4,
            learning_rate=0.1,
            n_estimators=int(1.2*model.best_iteration_),
            min_child_weight=1,
            min_data_in_leaf=50,
            subsample = 0.8,
            feature_fraction=0.4,
            bagging_seed=1,
            reg_alpha=1,
            reg_lambda=1,  # 此处不改了
            min_sum_hessian_in_leaf=0.01,
            random_state=1212
        )
        model2.fit(tmp[features], tmp[label])
    else:
        model = lgb.LGBMRegressor(
            boosting="gbdt",
            max_depth=5,
            learning_rate=0.1,
            n_estimators=10000,
            min_child_weight=1,
            min_data_in_leaf=60,
            subsample = 0.8,
            feature_fraction=0.4,
            bagging_seed=1,
            reg_alpha=0.1,
            reg_lambda=0.1,  # 此处不改了
            min_sum_hessian_in_leaf=0.01,
            random_state=1212
        )

        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['mse'],
              early_stopping_rounds=150, verbose=200)
        test_pred = model.predict(X_test)
        score_list.append(np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(test_pred))))
        print(f"score_list = {score_list}")
        model2 = lgb.LGBMRegressor(
            boosting="gbdt",
            max_depth=5,
            learning_rate=0.11,
            n_estimators=int(1.2*model.best_iteration_),
            min_child_weight=1,
            min_data_in_leaf=60,
            subsample = 0.7,
            feature_fraction=0.4,
            bagging_seed=1,
            reg_alpha=0.1,
            reg_lambda=0.1,  # 此处不改了
            min_sum_hessian_in_leaf=0.01,
            random_state=1212
        )
        model2.fit(tmp[features], tmp[label])

    test[label] = np.expm1(model2.predict(test[features]))

35068
11.102353086933864 6.703789159570189
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.01, min_child_weight=1 will be ignored. Current value: min_sum_hessian_in_leaf=0.01
Training until validation scores don't improve for 150 rounds
[200]	training's l2: 0.00848376	valid_1's l2: 0.012306
[400]	training's l2: 0.00638753	valid_1's l2: 0.011665
[600]	training's l2: 0.00518676	valid_1's l2: 0.0114179
[800]	training's l2: 0.00436554	valid_1's l2: 0.0113176
[1000]	training's l2: 0.00372538	valid_1's l2: 0.0112709
[1200]	training's l2: 0.00326733	valid_1's l2: 0.0112415
[1400]	training's l2: 0.002925	valid_1's l2: 0.0112377
Early stoppi

In [8]:
loss = np.mean(score_list)
1000/(1+loss)

0.7863277985129252

In [9]:
test[['time'] + labels].to_csv("./res/base_line.csv", index=False)

In [10]:
model.best_iteration_

204